In [ ]:
pip list

: 

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

: 

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data.csv')
df1

: 

In [ ]:
df1.groupby('area_type')['area_type'].agg('count')

: 

In [ ]:
df2 = df1.drop(['area_type', 'society', 'balcony', 'availability'], axis = 'columns')
df2.head()

: 

In [ ]:
df2.isnull().sum()

: 

In [ ]:
df2.shape

: 

In [ ]:
df3 = df2.dropna()
df3.isnull().sum()

: 

To Check the values


In [ ]:
df3['size'].unique()

: 

In [ ]:
df3['bhk'] = df3['size'].apply( lambda x:int(x.split(' ')[0]))

: 

In [ ]:
df3[df3.bhk > 20]

: 

In [ ]:
df3.total_sqft.unique()

: 

In [ ]:
def is_float(x):
  try:
    float(x)
  except:
    return False
  return True

: 

## WE use the ~ to get the values that are not true

In [ ]:
df3[~df3['total_sqft'].apply(is_float)].head(20)

: 

so we are going to take the average of the values for input total square feet like 1000-1200 and we are going to delete the rows that are in other unit .

In [ ]:
def convert_sqft_to_num(x):
  tokens = x.split('-')
  if len(tokens) == 2:
    return (float(tokens[0])+float(tokens[1]))/2
  try :
    return float(x)
  except:
    return None

: 

In [ ]:
convert_sqft_to_num('1000 - 1200')

: 

In [ ]:
convert_sqft_to_num('30Acres	')

: 

In [ ]:
df4 = df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(convert_sqft_to_num)
df4.shape

: 

In [ ]:
df4.loc[30]

: 

In [ ]:
df4.head()

: 

In [ ]:
df5 = df4.copy()
df5['price_per_sqft'] = df5['price'] * 100000 / df5['total_sqft']
df5.head()

: 

In [ ]:
len(df5.location.unique())

: 

In [ ]:
df5.location = df5.location.apply(lambda x:x.strip())

location_stats = df5.groupby('location')['location'].agg('count').sort_values(ascending =False)
location_stats

: 

In [ ]:
len(location_stats[location_stats <=10])

: 

In [ ]:
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

: 

In [ ]:
len(df5.location.unique())

: 

In [ ]:
df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())

: 

In [ ]:
df5.head(10)

: 

In [ ]:
df5[df5.total_sqft/df5.bhk < 300].head()

: 

In [ ]:
df6 = df5[~(df5.total_sqft/df5.bhk < 300)]
df6.shape

: 

In [ ]:
df6.price_per_sqft.describe()

: 

In [ ]:
def remove_pps_outliers(df):
  df_out = pd.DataFrame()
  for key, subdf in df.groupby('location'):
    m = np.mean(subdf.price_per_sqft)
    st = np.std(subdf.price_per_sqft)
    reduced_df = subdf[(subdf.price_per_sqft > (m-st)) & (subdf.price_per_sqft <= (m+st))]
    df_out = pd.concat([df_out, reduced_df], ignore_index = True)
  return df_out

df7 = remove_pps_outliers(df6)
df7.shape

: 

In [ ]:
def plot_scatter_chart(df,location):
  bhk2 = df[(df.location == location) & (df.bhk==2)]
  bhk3 = df[(df.location == location) & (df.bhk==3)]
  plt.scatter(bhk2.total_sqft, bhk2.price_per_sqft, color= 'blue', label = '2BHK' , s = 50)
  plt.scatter(bhk3.total_sqft, bhk3.price_per_sqft, color= 'green', label = '3BHK' , s = 50, marker = "+")
  plt.xlabel("Total Square Feet Area")
  plt.ylabel("Price Per Square Feet")
  plt.title(location)
  plt.legend()


plot_scatter_chart(df7, "Hebbal")

: 

In [ ]:
def remove_bhk_outliers (df):
  exclude_indices = np.array([])
  for location, location_df in df.groupby('location'):
    bhk_stats = {}
    for bhk, bhk_df in location_df.groupby('bhk'):
      bhk_stats[bhk] = {
          'mean': np.mean (bhk_df.price_per_sqft),
          'std': np.std(bhk_df.price_per_sqft),
          'count': bhk_df.shape[0]
          }
    for bhk, bhk_df in location_df.groupby('bhk'):
      stats = bhk_stats.get(bhk-1)
      if stats and stats['count']>5:
        exclude_indices = np.append(exclude_indices, bhk_df [bhk_df.price_per_sqft<(stats['mean'])].index.values)
  return df.drop(exclude_indices, axis='index')

df8 = remove_bhk_outliers (df7)
df8.shape

: 

In [ ]:
plot_scatter_chart(df8,"Hebbal")

: 

In [ ]:
import matplotlib
plt.hist(df8.price_per_sqft, rwidth = 0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

: 

In [ ]:
df8.bath.unique()

: 

In [ ]:
df8[df8.bath> 10]

: 

In [ ]:
plt.hist(df8.bath, rwidth=0.8)
plt.xlabel("Number of bathroom")
plt.ylabel("Count")

: 

In [ ]:
df8[df8.bath>df8.bhk+2]

: 

In [ ]:
df9 = df8[df8.bath<df8.bhk+2]
df9.shape

: 

In [ ]:
df10 = df9.drop(['size', 'price_per_sqft'], axis='columns')
df10.head(3)

: 

In [ ]:
dummies = pd.get_dummies(df10.location)
df11 = pd.concat([df10,dummies.drop('other', axis='columns')], axis = 'columns')
df11.head()

: 

In [ ]:
df12 = df11.drop('location', axis = 'columns')
df12.head(3)

: 

In [ ]:
df12.shape

: 

In [ ]:
X = df12.drop('price', axis='columns')
X.head()

: 

In [ ]:
Y = df12.price
Y.head()

: 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, random_state = 10)

: 

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, Y_train)
lr_model.score(X_test,Y_test)

: 

In [ ]:
from sklearn.model_selection import ShuffleSplit, cross_val_score

cv = ShuffleSplit(n_splits = 5, test_size=0.2, random_state = 0)
cross_val_score(LinearRegression(), X,Y,cv=cv)

: 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X, Y):
    algos = {
        'linear_regression': {
            'model': LinearRegression(),
            'params': {}
        },

        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1, 2],
                'selection': ['random', 'cyclic']
            }
        },

        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse', 'friedman_mse'],
                'splitter': ['best', 'random']
            }
        }
    }

    scores = []
    cv = ShuffleSplit(n_splits = 5,test_size = 0.2,random_state=0)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X, Y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns = ['model', 'best_score', 'best_params'])

find_best_model_using_gridsearchcv(X,Y)

: 

In [ ]:
import pickle

with open('bangalore_home_price_model.pickle', 'wb') as f:
  pickle.dump(lr_model, f)

: 

In [ ]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}

with open('columns.json','w') as f:
  f.write(json.dumps(columns))


: 

: 